# Neurosnap API Tutorial
[![Neurosnap Header](https://raw.githubusercontent.com/NeurosnapInc/neurosnap/refs/heads/main/assets/header.webp)](https://neurosnap.ai/)

In this tutorial, we demonstrate how to use the NeurosnapAPI class and associated methods from the [Neurosnap tools package](https://github.com/NeurosnapInc/neurosnap/). This guide will walk you through the various functionalities provided by the Neurosnap API, helping you understand how to interact with Neurosnap services, manage jobs, and work with team information.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NeurosnapInc/neurosnap/blob/main/example_notebooks/api.ipynb)

#### Part 0. Installing Dependencies on Colab

In [1]:
import subprocess
import sys

if 'google.colab' in sys.modules:
    subprocess.run("pip install -U --no-cache-dir git+https://github.com/NeurosnapInc/neurosnap.git", shell=True, check=True, executable="/bin/bash")

#### Part 1. `NeurosnapAPI` Class: Connecting to Neurosnap
The `NeurosnapAPI` class allows you to connect to the Neurosnap platform using your API key. Upon creating an instance of this class, it automatically checks the validity of your key by making a request to the Neurosnap API. If the key is invalid, an error will be raised, prompting you to verify your credentials. This authentication step is essential for accessing the platform’s features, such as submitting jobs and retrieving data.
You can generate and delete API keys by visiting the [Neurosnap Overview page](https://neurosnap.ai/overview) under the "API Key" section.

In [2]:
import os

import pandas as pd
from neurosnap.api import NeurosnapAPI
from neurosnap.protein import Protein

api_key = os.environ.get('ns_key')
api = NeurosnapAPI(api_key) # replace with your API key


[*] Successfully connected to the Neurosnap API.
 - For information visit https://neurosnap.ai/blog/post/66b00dacec3f2aa9b4be703a
 - For support visit https://neurosnap.ai/support
 - For bug reports visit https://github.com/NeurosnapInc/neurosnap


#### Part 2. `get_services` Method: Retrieving Neurosnap Services
The get_services method allows you to fetch a list of all available services on the Neurosnap platform. This function gives you the flexibility to print the services in different formats or simply retrieve them for further use.

Functionality:
- Fetch Services: It retrieves a list of available services from Neurosnap's API and returns it as a list of dictionaries.
- Optional Printing: You can optionally print the services in two formats:
    - "table": Displays services in a table format showing key information (title, beta status, link).
    - "json": Prints the raw JSON response, formatted for readability.
    - None: By default, the function will not print anything but will return the data.

In [3]:
services = api.get_services(format_type="table")

Title                                         Beta    Link
--------------------------------------------  ------  -------------------------------------------------------------------------------
NeuroFold                                     False   https://neurosnap.ai/service/NeuroFold
AlphaFold2                                    False   https://neurosnap.ai/service/AlphaFold2
DiffDock-L                                    False   https://neurosnap.ai/service/DiffDock-L
DynamicBind                                   False   https://neurosnap.ai/service/DynamicBind
RFdiffusion-v2                                False   https://neurosnap.ai/service/RFdiffusion-v2
ColabDock                                     False   https://neurosnap.ai/service/ColabDock
RoseTTAFold2                                  False   https://neurosnap.ai/service/RoseTTAFold2
RoseTTAFold All-Atom                          False   https://neurosnap.ai/service/RoseTTAFold%20All-Atom
PocketFlow                            

#### Part 3. Job Management Methods in NeurosnapAPI
The following methods allow you to manage and interact with jobs submitted to Neurosnap. These methods cover everything from submitting new jobs, checking job status, and retrieving job files, to managing notes and sharing functionality.

`get_jobs`: Retrieve Submitted Jobs
This method fetches a list of all submitted jobs from Neurosnap.

- Fetch Jobs: Retrieves submitted jobs as a list of dictionaries.
- Optional Printing:
    - "table": Displays jobs in a table format.
    - "json": Prints the jobs in formatted JSON.
    - None: No printing by default, just returns the job data.

In [4]:
jobs = api.get_jobs(format_type="table")

Status     Job ID                    User ID                   Service                        Submitted    Runtime    Note    Shared                     Team ID               Pipeline ID    Pipeline Node ID    Pipeline Item Index
---------  ------------------------  ------------------------  -----------------------------  -----------  ---------  ------  --------  ------------------------  ------------------------  ------------------  ---------------------
completed  6702feb00db6770388b10e34  66c61493dfaceccb0cca0af1  ProteinMPNN                    2024-10-07   9s                 False     000000000000000000000000  000000000000000000000000                   0                      0
completed  6702fe220db6770388b10e33  66c61493dfaceccb0cca0af1  ProteinMPNN                    2024-10-07   15s                False     000000000000000000000000  000000000000000000000000                   0                      0
completed  670037140db6770388b10b59  66c61493dfaceccb0cca0af1  RoseTTAFold2     

`submit_job`: Submit a New Job
This method submits a new job to Neurosnap.

- Service Name: The Neurosnap service to run.
- Files: A dictionary mapping file names to file paths.
- Config: Any parameters required by the service.
- Job ID: Returns the unique job ID upon successful submission.

For this example, we will use the ProteinMPNN service. To ensure proper file and configuration setup for the service you intend to use, we recommend checking the service page on Neurosnap for detailed information about required inputs.

For instance, the [ProteinMPNN service page](https://neurosnap.ai/service/ProteinMPNN) specifies required files and configuration parameters, such as input structure, number of sequences, and sampling temperature.

In [8]:
prot = Protein("A0A411PAU0") # loading a protein
prot.save("./A0A411PAU0.pdb") # saving the protein structure to a PDB file
files = { # specify the files to be submitted
    "Input Structure": "./A0A411PAU0.pdb",
}
config = { # specify the configuration
    'Homo-oligomer': 'false',
    'Fixed Positions': '',
    'Invert Selection': 'false',
    'Number Sequences': '100',
    'Sampling Temperature': '0.15',
    'Model Type': 'v_48_020',
    'Model Version': 'original',
    'Excluded Amino Acids': ''
}

job_id = api.submit_job("ProteinMPNN", files, config)

[*] Found matching structure in AF-DB, downloading and using that.


`get_job_status`: Check Job Status
This method retrieves the status of a job by its ID.

- Job ID: Provide the ID of the job.
- Status Retrieval: Returns the current job status (e.g., "pending," "running," "completed").

In [9]:
api.get_job_status(job_id)

'completed'

`get_job_files`: Retrieve Files from a Completed Job
This method fetches all files from a completed job.

- Job ID & File Type: Specify the job ID and the type of files to retrieve (e.g., "in" for input files, "out" for output files).
- Optional Share ID: If the job is shared, include the share ID. (for shared jobs only that are not submitted by you) 
- Optional Printing:
    - "table": Prints file details in table format.
    - "json": Prints the files in formatted JSON.
    - None: No printing by default, just returns the file data.

In [10]:
job_files = api.get_job_files(job_id, file_type='out', format_type='table')

File Name                File Size
-----------------------  -----------
pssm_conditional.json    56.65 KB
pssm_unconditional.json  56.55 KB
results.csv              13.96 KB


`get_job_file`: Download a Specific File
This method allows you to download a specific file from a job.

- Job ID & File Name: Provide the job ID, file type, and file name.
- Save Path: Specify where the file should be saved locally.
- Optional Share ID: If the job is shared, include the share ID. (for shared jobs only that are not submitted by you) 

In [11]:
api.get_job_file(job_id, 'out', 'results.csv', save_path='./results.csv')

File saved to ./results.csv


('./results.csv', True)

For example, the `results.csv` file is the results of our ProteinMPNN job.

In [12]:
df = pd.read_csv('results.csv')
display(df.head())

,rank,score,seqid,seq
0,1,0.814,0.386,EVVLTESGGGTVAAGGSTTLTCTASTPVFNSSSATWAWWAQKAGGP...
1,2,0.815,0.394,SVVITESGGGTVAAGGSVTLTCTASTPAFNSSTATWSWWRQKAGGP...
2,3,0.816,0.402,ALVLTESGGGTVAAGGSTTLTCTASSSAFNSSTATWAWWAQKPGGP...
3,4,0.824,0.402,SITLTESGGGTVAAGGSTTLTCTASVPAFNSSSATWAWVRQKPGGP...
4,5,0.831,0.417,ELVLTESGGGTVAEGGSVTLTCTASKSAFTSSTATWGWLRQTPGGP...


`set_job_note`: Add a Note to a Job
This method lets you attach a note to a job for easier management.

- Job ID: Specify the job to attach the note to.
- Note: The text note you want to add.

In [13]:
api.set_job_note(job_id, 'Hello World!')

Note set successfully for job ID 67041e980db6770388b11028.


`set_job_share`: Enable Job Sharing
This method makes a job publicly accessible by enabling sharing.

- Job ID: Provide the job ID to enable sharing.
- Share ID: The method returns the share ID, which can be used to access the job publicly.

In [14]:
share_id = api.set_job_share(job_id)

print(f"https://neurosnap.ai/job/{job_id}?share={share_id}") # Public link to the job

https://neurosnap.ai/job/67041e980db6770388b11028?share=67041ec30db6770388b11029


`delete_job_share`: Disable Job Sharing
This method disables the sharing feature and makes a job private.

- Job ID: Provide the job ID to make it private.
- Action: The job is no longer publicly accessible.

In [15]:
api.delete_job_share(job_id)

Job ID 67041e980db6770388b11028 is now private.


#### Part 4. Team Collaboration
The following methods allow you to manage and retrieve information about your Neurosnap team, including team details and jobs submitted by team members.

`get_team_info`: Fetch Team Information
This method retrieves your team's information if you are part of a Neurosnap Team.

- Format Type: Optional parameter to specify the output format. You can choose from:
    - table: Displays team information in a tabular format.
    - json: Displays team information in JSON format.
    - None: No printing by default, just returns the data.
- Returns: A dictionary containing the team information.

In [ ]:
team_info = api.get_team_info(format_type="table")

`get_team_jobs`: Fetch Team Jobs
This method retrieves all the jobs submitted by all members of your Neurosnap Team.

- Format Type: Optional parameter to specify the output format. You can choose from:
    - table: Displays the list of jobs in a tabular format.
    - json: Displays the list of jobs in JSON format.
    - None: No printing by default, just returns the data.
- Returns: A list of dictionaries representing jobs submitted by team members. 

In [ ]:
team_jobs = api.get_team_jobs(format_type="table")